# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
# import graphlab
import math
import string
import pandas as pd
import numpy as np

# Data preperation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
dtype_dict = {'name': str, 'review': str, 'rating': int}

In [3]:
products = pd.read_csv('amazon_baby.csv', dtype=dtype_dict)
# products = pd.read_csv('amazon_baby.csv')

Now, let us see a preview of what the dataset looks like.

In [4]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


In [5]:
products['review'][1]

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [6]:
products.ix[269]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [7]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
text = "lkmsdc'sdfs'sdcca'"
text.translate(None, string.punctuation)

'lkmsdcsdfssdcca'

In [9]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

def count_words(text, vocabulary=None):
    text_list = text.lower().split()
    my_dict = {}
    if vocabulary == None:
        for item in text_list:
            if item in my_dict:
                my_dict[item] += 1
            else:
                my_dict[item] = 1
    else:
        for item in text_list:
            if item in vocabulary:
                if item in my_dict:
                    my_dict[item] += 1
                else:
                    my_dict[item] = 1        
    return my_dict

# products['review'] = products['review'].fillna('')  # fill in N/A's in the review column
products = products.fillna({'review':''})  # fill in N/A's in the review column (cooler way)
products['review_clean'] = products['review'].apply(remove_punctuation)

products['word_count'] = products['review_clean'].apply(count_words)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [10]:
products.ix[269]['word_count']

{'a': 1, 'favorite': 1, 'house': 1, 'in': 1, 'our': 1}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [11]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [12]:
products.loc[:, 'sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

,name,review,rating,review_clean,word_count,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{u'and': 3, u'love': 1, u'it': 3, u'highly': 1...",1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"{u'and': 2, u'quilt': 1, u'it': 1, u'comfortab...",1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"{u'and': 3, u'ingenious': 1, u'love': 2, u'is'...",1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"{u'and': 2, u'this': 1, u'all': 2, u'help': 1,...",1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,"{u'and': 2, u'cute': 1, u'would': 1, u'help': ...",1
6,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4,Lovely book its bound tightly so you may not b...,"{u'shop': 1, u'noble': 1, u'is': 1, u'bound': ...",1
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5,Perfect for new parents We were able to keep t...,"{u'and': 2, u'all': 1, u'would': 1, u'when': 1...",1
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...,"{u'and': 1, u'fantastic': 1, u'help': 1, u'giv...",1
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,"{u'all': 1, u'because': 1, u'another': 1, u'in...",1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,"{u'all': 2, u'nannys': 1, u'just': 1, u'sleep'...",1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [13]:
products_old = products.copy()

In [14]:
products = products.reset_index(drop=True)
products

,name,review,rating,review_clean,word_count,sentiment
0,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,"{u'and': 3, u'love': 1, u'it': 3, u'highly': 1...",1
1,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,"{u'and': 2, u'quilt': 1, u'it': 1, u'comfortab...",1
2,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,"{u'and': 3, u'ingenious': 1, u'love': 2, u'is'...",1
3,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,"{u'and': 2, u'this': 1, u'all': 2, u'help': 1,...",1
4,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,"{u'and': 2, u'cute': 1, u'would': 1, u'help': ...",1
5,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4,Lovely book its bound tightly so you may not b...,"{u'shop': 1, u'noble': 1, u'is': 1, u'bound': ...",1
6,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5,Perfect for new parents We were able to keep t...,"{u'and': 2, u'all': 1, u'would': 1, u'when': 1...",1
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...,"{u'and': 1, u'fantastic': 1, u'help': 1, u'giv...",1
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,"{u'all': 1, u'because': 1, u'another': 1, u'in...",1
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,"{u'all': 2, u'nannys': 1, u'just': 1, u'sleep'...",1


In [15]:
test_indices = pd.read_json('module-2-assignment-test-idx.json')
train_indices = pd.read_json('module-2-assignment-train-idx.json')
train_data = products.iloc[train_indices[0], :] 
test_data = products.iloc[test_indices[0], :]

In [16]:
# train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)
print 'split percent = {}'.format(len(train_data)/(len(train_data)+len(test_data)))

133416
33336
split percent = 0.800086355786


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [17]:
print 'abc\bdef'

abcdef


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [19]:
from sklearn.linear_model import LogisticRegression

In [20]:
sentiment_model = LogisticRegression()
sentiment_model = sentiment_model.fit(train_matrix, train_data['sentiment'])

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [21]:
print sentiment_model.coef_[0]
print sentiment_model.intercept_

[ -1.23446855e+00   2.31508242e-04   2.55829582e-02 ...,   1.11803871e-02
   3.24141074e-03  -7.60256923e-05]
[ 1.37982065]


In [22]:
len(sentiment_model.coef_[0])

121712

In [23]:
sentiment_model_coef = pd.Series(sentiment_model.coef_[0], index=np.asarray(vectorizer.get_feature_names()))
sentiment_model_coef

0                     -1.234469e+00
00                     2.315082e-04
000                    2.558296e-02
0001                   6.554095e-03
001                    5.319143e-05
001cm                  8.599205e-07
002                    2.355377e-03
01                     2.597168e-01
010                    2.493832e-01
010204                -2.064954e-03
0104                   5.521395e-02
010613do              -1.379151e-04
01082013              -2.209589e-01
012                    2.786307e-01
012010                -7.289632e-04
012013                 7.652382e-06
01202012               1.788992e-05
01252013              -4.941293e-03
01302012my            -1.290813e-02
01312009               9.807963e-05
015a                   8.067906e-02
017                   -8.006455e-02
0182196                3.589968e-03
02                     9.819889e-02
020                    1.182685e-02
02000z                 2.294598e-01
02060                  8.793128e-02
0207                  -2.835

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [24]:
num_positive_weights = (sentiment_model.coef_[0] > 0).sum() + (sentiment_model.intercept_[0] > 0).sum()
num_negative_weights = (sentiment_model.coef_[0] < 0).sum()

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 85704 
Number of negative weights: 36009 


**Quiz question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [25]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

53    5
64    2
82    1
Name: rating, dtype: int64


,name,review,rating,review_clean,word_count,sentiment
53,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,"{u'and': 2, u'all': 1, u'love': 1, u'purchased...",1
64,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend The deca...,"{u'and': 1, u'almost': 1, u'for': 1, u'decals'...",-1
82,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,"{u'all': 1, u'on': 1, u'girls': 1, u'is': 1, u...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [26]:
sample_test_data.iloc[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [27]:
sample_test_data.iloc[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [28]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  5.58323494  -3.11950259 -10.3943161 ]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [29]:
def predict_labels(scores):
    labels = (scores > 1).astype(int)
    labels[labels < 1] = -1
    return labels

In [30]:
print predict_labels(scores)

[ 1 -1 -1]


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [31]:
print "Class predictions according to sklearn:" 
print sentiment_model.predict(sample_test_matrix)

Class predictions according to sklearn:
[ 1 -1 -1]


In [32]:
t1 = predict_labels(scores)
t2 = sentiment_model.predict(sample_test_matrix)

# for i in xrange(len(t1)):
#     print t1[i] and t2[i]

**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [33]:
def probability_prediction(scores):
    return 1/ (1 + np.exp(-scores))

In [34]:
probability_prediction(scores)

array([  9.96253706e-01,   4.23099222e-02,   3.06050139e-05])

**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [35]:
print "Class predictions according to sklearn:" 
print "\tP(y=-1|x,w) \t P(y=1|x,w)"
print sentiment_model.predict_proba(sample_test_matrix)

Class predictions according to sklearn:
	P(y=-1|x,w) 	 P(y=1|x,w)
[[  3.74629373e-03   9.96253706e-01]
 [  9.57690078e-01   4.23099222e-02]
 [  9.99969395e-01   3.06050139e-05]]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [36]:
test_probs_allclasses = sentiment_model.predict_proba(test_matrix)
test_probs = test_probs_allclasses[:,1]

In [37]:
test_data['preds'] = test_probs

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [38]:
test_data

,name,review,rating,review_clean,word_count,sentiment,preds
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,"{u'all': 1, u'because': 1, u'another': 1, u'in...",1,0.778836
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,"{u'all': 2, u'nannys': 1, u'just': 1, u'sleep'...",1,0.999999
14,Nature's Lullabies First Year Sticker Calendar,"I love this little calender, you can keep trac...",5,I love this little calender you can keep track...,"{u'and': 1, u'the': 2, u'little': 1, u'love': ...",1,0.934677
18,Nature's Lullabies Second Year Sticker Calendar,I had a hard time finding a second year calend...,5,I had a hard time finding a second year calend...,"{u'and': 3, u'all': 1, u'later': 1, u'referenc...",1,0.999978
24,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,One of babys first and favorite books and it i...,"{u'and': 2, u'because': 1, u'just': 1, u'less'...",1,0.980064
31,"Lamaze Peekaboo, I Love You",My son loved this book as an infant. It was p...,5,My son loved this book as an infant It was pe...,"{u'infant': 1, u'being': 1, u'all': 1, u'cours...",1,0.999949
32,"Lamaze Peekaboo, I Love You",Our baby loves this book & has loved it for a ...,5,Our baby loves this book has loved it for a w...,"{u'and': 1, u'own': 1, u'it': 3, u'our': 1, u'...",1,0.998711
36,"SoftPlay Giggle Jiggle Funbook, Happy Bear",This bear is absolutely adorable and I would g...,2,This bear is absolutely adorable and I would g...,"{u'and': 3, u'rating': 1, u'disappointmentits'...",-1,0.806427
38,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,I bought two for recent baby showers! The boo...,5,I bought two for recent baby showers The book...,"{u'and': 2, u'beautiful': 1, u'love': 1, u'elm...",1,0.998469
50,Baby's First Year Undated Wall Calendar with S...,I searched high and low for a first year calen...,5,I searched high and low for a first year calen...,"{u'and': 4, u'remembering': 1, u'personalizati...",1,0.997330


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [39]:
test_data.sort_values(by=['preds'], ascending=False).head(20)

,name,review,rating,review_clean,word_count,sentiment,preds
108269,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,"{u'all': 4, u'excise': 1, u'just': 2, u'money'...",1,1
45751,"P'Kolino Silly Soft Seating in Tias, Green",I've purchased both the P'Kolino Little Reader...,4,Ive purchased both the PKolino Little Reader C...,"{u'rating': 1, u'since': 1, u'particularly': 1...",1,1
72823,"Simple Wishes Hands-Free Breastpump Bra, Pink,...","I just tried this hands free breastpump bra, a...",5,I just tried this hands free breastpump bra an...,"{u'all': 1, u'spaced': 1, u'just': 4, u'being'...",1,1
124541,Graco Pack 'n Play Element Playard - Flint,My husband and I assembled this Pack n' Play l...,4,My husband and I assembled this Pack n Play la...,"{u'all': 2, u'45': 1, u'just': 1, u'organic': ...",1,1
134472,"Baby Jogger City Mini GT Single Stroller, Shad...","Amazing, Love, Love, Love it !!! All 5 STARS a...",5,Amazing Love Love Love it All 5 STARS all the...,"{u'all': 3, u'just': 1, u'over': 2, u'saves': ...",1,1
152721,Buttons Cloth Diaper Cover - One Size - 8 Colo...,"We are big Best Bottoms fans here, but I wante...",4,We are big Best Bottoms fans here but I wanted...,"{u'all': 2, u'just': 4, u'less': 1, u'being': ...",1,1
127998,"Diono RadianRXT Convertible Car Seat, Plum",I bought this seat for my tall (38in) and thin...,5,I bought this seat for my tall 38in and thin 2...,"{u'all': 5, u'consider': 1, u'focus': 2, u'wra...",1,1
121469,"Britax 2012 B-Agile Stroller, Red",[I got this stroller for my daughter prior to ...,4,I got this stroller for my daughter prior to t...,"{u'hands': 1, u'all': 5, u'allowed': 1, u'birt...",1,1
47873,Evenflo X Sport Plus Convenience Stroller - Ch...,After seeing this in Parent's Magazine and rea...,5,After seeing this in Parents Magazine and read...,"{u'bending': 1, u'basketserenity': 1, u'all': ...",1,1
90999,"Infantino Wrap and Tie Baby Carrier, Black Blu...",I bought this carrier when my daughter was abo...,5,I bought this carrier when my daughter was abo...,"{u'cute': 1, u'all': 2, u'just': 1, u'less': 1...",1,1


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

In [41]:
test_data.sort_values(by=['preds'], ascending=True).head(20)

,name,review,rating,review_clean,word_count,sentiment,preds
14637,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,"{u'fishstarfish': 1, u'all': 5, u'managed': 1,...",-1,9.204188e-16
109216,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,"{u'all': 2, u'contacted': 1, u'being': 1, u'wh...",-1,1.964535e-15
70004,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,"{u'just': 1, u'whoever': 1, u'being': 1, u'sou...",-1,8.614861e-14
44284,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,"{u'all': 2, u'forget': 1, u'grateful': 1, u'mi...",-1,1.448999e-13
141135,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,"{u'all': 4, u'reviewers': 1, u'beware': 1, u'e...",-1,1.495178e-13
85925,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,"{u'all': 3, u'go': 1, u'powercycling': 1, u'se...",-1,4.793706e-13
48396,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,"{u'all': 3, u'think': 1, u'being': 1, u'over':...",-1,3.331440e-11
73893,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,"{u'all': 1, u'just': 1, u'money': 1, u'random'...",-1,4.378382e-11
103564,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,"{u'currently': 1, u'contacted': 2, u'less': 1,...",-1,9.345721e-11
9768,Philips AVENT Newborn Starter Set,"It's 3am in the morning and needless to say, t...",1,Its 3am in the morning and needless to say thi...,"{u'all': 3, u'just': 2, u'money': 1, u'over': ...",-1,1.121744e-10


## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [188]:
def get_classification_accuracy(model, data, true_labels, words_subset = False):
    # First get the predictions
    if not(words_subset):
        test_data_matrix = vectorizer.transform(data['review_clean'])
    else:
        test_data_matrix = vectorizer_word_subset.transform(data['review_clean'])
    preds = model.predict(test_data_matrix)
    
    # Compute the number of correctly classified examples
    correctly_classified = sum(preds == true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = correctly_classified/len(data)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [167]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.93226541876649871

In [168]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.96754512202434495

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [169]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [170]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [171]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

In [206]:
vectorizer_word_subset.get_feature_names()

['love',
 'great',
 'easy',
 'old',
 'little',
 'perfect',
 'loves',
 'well',
 'able',
 'car',
 'broke',
 'less',
 'even',
 'waste',
 'disappointed',
 'work',
 'product',
 'money',
 'would',
 'return']

In [179]:
train_data['word_count_subset'] = train_data['review_clean'].apply(count_words, vocabulary=significant_words)
test_data['word_count_subset'] = test_data['review_clean'].apply(count_words, vocabulary=significant_words)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Let's see what the first example of the dataset looks like:

In [181]:
train_data.iloc[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [182]:
print train_data.iloc[0]['word_count']

{'and': 3, 'love': 1, 'it': 3, 'highly': 1, 'osocozy': 1, 'bags': 1, 'disappointed': 1, 'moist': 1, 'does': 1, 'recommend': 1, 'was': 1, 'wipes': 1, 'early': 1, 'not': 2, 'now': 1, 'holder': 1, 'wipe': 1, 'keps': 1, 'wise': 1, 'i': 1, 'planet': 1, 'leak': 1, 'my': 2, 'came': 1}


Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [184]:
print train_data.iloc[0]['word_count_subset']

{'love': 1, 'disappointed': 1}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [193]:
simple_model = LogisticRegression()
simple_model = simple_model.fit(train_matrix_word_subset, train_data['sentiment'])

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [194]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'], words_subset = True)

0.86936045116390692

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [195]:
simple_model.coef_

array([[ 1.36368976,  0.94399959,  1.19253827,  0.08551278,  0.52018576,
         1.50981248,  1.67307389,  0.50376046,  0.19090857,  0.05885467,
        -1.65157634, -0.20956286, -0.51137963, -2.03369861, -2.34829822,
        -0.62116877, -0.32055624, -0.89803074, -0.36216674, -2.10933109]])

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [211]:
np.asarray(vectorizer_word_subset.get_feature_names())

array(['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves',
       'well', 'able', 'car', 'broke', 'less', 'even', 'waste',
       'disappointed', 'work', 'product', 'money', 'would', 'return'], 
      dtype='|S12')

In [202]:
print np.sort(simple_model.coef_)
print simple_model.intercept_
print len(simple_model.coef_[0])+1

[[-2.34829822 -2.10933109 -2.03369861 -1.65157634 -0.89803074 -0.62116877
  -0.51137963 -0.36216674 -0.32055624 -0.20956286  0.05885467  0.08551278
   0.19090857  0.50376046  0.52018576  0.94399959  1.19253827  1.36368976
   1.50981248  1.67307389]]
[ 1.29937369]
21


**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [218]:
simple_model_coef = pd.Series(simple_model.coef_[0], index=np.asarray(vectorizer_word_subset.get_feature_names()))
simple_model_coef

love            1.363690
great           0.944000
easy            1.192538
old             0.085513
little          0.520186
perfect         1.509812
loves           1.673074
well            0.503760
able            0.190909
car             0.058855
broke          -1.651576
less           -0.209563
even           -0.511380
waste          -2.033699
disappointed   -2.348298
work           -0.621169
product        -0.320556
money          -0.898031
would          -0.362167
return         -2.109331
dtype: float64

In [229]:
simple_model_coef['love']

1.3636897593110029

In [232]:
condition = True
for word in significant_words:
    if simple_model_coef[word] > 0:
        if sentiment_model_coef[word] < 0:
            condition = False
            break

print condition

True


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [233]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.96754512202434495

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [234]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'], words_subset = True)

0.8668225700065959

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [235]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.93226541876649871

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [236]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'], words_subset = True)

0.86936045116390692

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [237]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [238]:
sum(test_data['sentiment']==1)/len(test_data)

0.84278257739380846

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?

Yes, the sentiment_model definitely better than the majority class classifier